# Day 3: Gear Ratios

In [100]:
import re
import pandas as pd
import numpy as np
from functools import reduce

# Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [101]:
def part_number(line):

    # Separate characters
    l = re.findall(r'.', line)

    # Find all digits and their indices in the line
    matches = re.finditer(r'\d+', line)
    result = [(match.group(), match.start()) for match in matches]

    for num, idx in result:
        # Replace all digits of num by num
        l[idx:idx+len(num)] = [int(num)] * (len(num))
           
    # Create dataframe
    dt = pd.DataFrame(l).transpose()     
    # Replace elements different from the integer or period by pi
    dt = dt.map(lambda x: x if x == '.' or type(x) == int else "π")
    return dt

In [102]:
def sum_about_pi(res,df0,df1,df2,debug):

    # Join and fill NaN by 0 to avoid first and last line mixup.
    df = pd.concat([df0,df1,df2], axis=0).fillna(0)

    # Position of pi in middle-dataframe
    pi_loc = np.where(df.iloc[1,:] == "π")[0]

    if debug:
        print("----------")
        print("DataFrame:")
        print(df)
        print("----------")
        print("Where is pi (in middle-row): " + str(pi_loc))

    # Store values to check if number is repeated
    val_00 = 0
    val_11 = 0
    val_22 = 0

    # Loop through all pi and sum neighbour values
    for pi in pi_loc:       
        # Positions: left, centre, right
        for e in range(pi-1, pi+2):

            # Store values
            val_0 = df.iloc[0,e] 
            val_1 = df.iloc[1,e]
            val_2 = df.iloc[2,e]
            
            if type(val_0) == int and val_0 != val_00:
                if debug:
                    print("**** Number in upper row: " + str(val_0))

                res += val_0

            if type(val_1) == int and val_1 != val_11:
                if debug:
                    print("**** Number in middle row: " + str(val_1))
            
                res += val_1

            if type(val_2) == int and val_2 != val_22:
                if debug:
                    print("**** Number in bottom row: " + str(val_2))
            
                res += val_2

            # Update values
            val_00 = val_0
            val_11 = val_1
            val_22 = val_2

    return res

In [103]:
def v2_sum_about_pi(res,df0,df1,df2,debug):

    # Join and fill NaN by 0 to avoid first and last line mixup.
    df = pd.concat([df0,df1,df2], axis=0).fillna(0)

    # Position of pi in middle-dataframe
    pi_loc = np.where(df.iloc[1,:] == "π")[0]

    if debug:
        print("----------")
        print("DataFrame:")
        print(df)
        print("----------")
        print("Where is pi (in middle-row): " + str(pi_loc))

    # Loop through all pi and sum neighbour values
    for pi in pi_loc:

        # Initialize list of values about pi
        my_list = (df.iloc[0, pi-1:pi+2].values.flatten().tolist() +
                    df.iloc[1, [pi-1,pi+1]].values.flatten().tolist() +
                    df.iloc[2, pi-1:pi+2].values.flatten().tolist())
        
        # Keep only one copy of each integer and remove consecutive positions
        non_con = [my_list[i] for i in range(len(my_list)) if i == 0 or my_list[i] != my_list[i-1] or type(my_list[i]) is not int]

        # Keep only the integers in the list
        part_nums = [x for x in non_con if isinstance(x, int)]

        if debug:
            print("Part numbers:")
            print(my_list)
            print("Non-consecutive numbers:")
            print(non_con)
            print("Part numbers:")
            print(part_nums)

        res += sum(part_nums)
            
    return res

In [111]:
debug = False
# Open the file in read mode
file_path = 'input_3.txt'  

# Initialize dataframe
da0 = pd.DataFrame(np.zeros((1, 1)))
da1 = pd.DataFrame(np.zeros((1, 1)))
da2 = pd.DataFrame(np.zeros((1, 1)))

# Solution
res = 0

with open(file_path, 'r') as file:
    for i in range(1,6):
        # Continue with all lines left
        for line in file:
            da2 = part_number(line)

            # Calculate sum number
            # res = v2_sum_about_pi(res,da0,da1,da2,debug)
            res = sum_about_pi(res,da0,da1,da2,debug)

            # Reorganize
            da0 = da1
            da1 = da2

# Last line
da3 = pd.DataFrame(np.zeros((1, 1)))

# Calculate sum number
# res = v2_sum_about_pi(res,da0,da1,da3,debug)
res = sum_about_pi(res,da0,da1,da3,debug)

print(res)

560670


# Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..
    
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?



In [112]:
def part_number2(line):

    # Separate characters
    l = re.findall(r'.', line)

    # Find all digits and their indices in the line
    matches = re.finditer(r'\d+', line)
    result = [(match.group(), match.start()) for match in matches]

    for num, idx in result:
        # Replace all digits of num by num
        l[idx:idx+len(num)] = [int(num)] * (len(num))
           
    # Create dataframe
    dt = pd.DataFrame(l).transpose()     

    return dt

In [107]:
def gear_ratio(res,df0,df1,df2,debug):

    # Count number of num about *
    count = 0
    mul_res = 1

    # Join and fill NaN by 0 to avoid first and last line mixup.
    df = pd.concat([df0,df1,df2], axis=0).fillna(".")

    # Position of * in middle-dataframe
    pi_loc = np.where(df.iloc[1,:] == "*")[0]

    if debug:
        print("----------")
        print("DataFrame:")
        print(df)
        print("----------")
        print("Where is * (in middle-row): " + str(pi_loc))

    # Store values to check if number is repeated
    val_00 = 0
    val_11 = 0
    val_22 = 0

    # Loop through all * and sum neighbour values
    for pi in pi_loc:       
        # Positions: left, centre, right
        for e in range(pi-1, pi+2):

            # Store values
            val_0 = df.iloc[0,e] 
            val_1 = df.iloc[1,e]
            val_2 = df.iloc[2,e]
            
            if type(val_0) == int and val_0 != val_00:

                # Count number
                count += 1
                if debug:
                    print("**** Count in upper row: " + str(val_0))

                mul_res *= val_0

            if type(val_1) == int and val_1 != val_11:

                # Count number
                count += 1
                if debug:
                    print("**** Count in middle row: " + str(val_1))

                mul_res *= val_1

            if type(val_2) == int and val_2 != val_22:

                # Count number
                count += 1
                if debug:
                    print("**** Count in bottom row: " + str(val_2))
                mul_res *= val_2

            # Update values
            val_00 = val_0
            val_11 = val_1
            val_22 = val_2
        
        # Impose condition of only two numbers about gear  
        if count == 2:
            if debug:
                print("**** Gear ratio: " + str(mul_res))

            # Add gear ratio
            res += mul_res
            # Reset param for next *
            count = 0
            mul_res = 1
            
        else:
            if debug:
                print("**** Gear ratio: " + "0")
            # Reset count for next *
            count = 0
            mul_res = 1
            
    return res

In [108]:
def v2_gear_ratio(res,df0,df1,df2,debug):
   
    # Join and fill NaN by 0 to avoid first and last line mixup.
    df = pd.concat([df0,df1,df2], axis=0).fillna(".")

    # Position of pi in middle-dataframe
    pi_loc = np.where(df.iloc[1,:] == "*")[0]

    if debug:
        print("----------")
        print("DataFrame:")
        print(df)
        print("----------")
        print("Where is pi (in middle-row): " + str(pi_loc))

    # Loop through all pi and sum neighbour values
    for pi in pi_loc:

        # Initialize list of values about pi
        my_list = (df.iloc[0, pi-1:pi+2].values.flatten().tolist() +
                    df.iloc[1, [pi-1,pi+1]].values.flatten().tolist() +
                    df.iloc[2, pi-1:pi+2].values.flatten().tolist())
        
        # Keep only one copy of each integer and remove consecutive positions
        non_con = [my_list[i] for i in range(len(my_list)) if i == 0 or my_list[i] != my_list[i-1] or type(my_list[i]) is not int]

        # Keep only the integers in the list
        ratios = [x for x in non_con if isinstance(x, int)]

        if len(ratios) == 2:
            if debug:
                print("Gear ratio:")
                print(ratios)
        
            res += reduce(lambda x, y: x * y, ratios)
            
    return res

In [110]:
debug = False
# Open the file in read mode
file_path = 'input_3.txt'  

# Initialize dataframe
da0 = pd.DataFrame({"": ['.']})
da1 = pd.DataFrame({"": ['.']})
da2 = pd.DataFrame({"": ['.']})

# Solution
res = 0

with open(file_path, 'r') as file:
    for i in range(1,6):
        # Continue with all lines left
        for line in file:
            da2 = part_number2(line)

            # Calculate sum number
            res = gear_ratio(res,da0,da1,da2,debug)
            # res = v2_gear_ratio(res,da0,da1,da2,debug)

            # Reorganize
            da0 = da1
            da1 = da2

# Last line
da3 = pd.DataFrame({"": ['.']})

# Calculate sum number
res = gear_ratio(res,da0,da1,da3,debug)
# res = v2_gear_ratio(res,da0,da1,da3,debug)

print(res)

91622824
